In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os

plt.style.use('fivethirtyeight')
sns.set_palette('muted')
pd.set_option('display.max_columns', None)

class NaturalGasStrategy:
    def __init__(self):
        self.price_data = None
        self.inventory_data = None
        self.hdd_data = None
        self.cdd_data = None
        self.positioning_data = None
        self.merged_data = None
        self.trading_data = None
        self.trades = []
        self.performance = {}
    
    def load_data(self, price_file, inventory_file, hdd_file, cdd_file, positioning_file):
        self.price_data = pd.read_csv(price_file)
        print(f"Loaded price data with {len(self.price_data)} rows")
        
        self.inventory_data = pd.read_csv(inventory_file)
        print(f"Loaded inventory data with {len(self.inventory_data)} rows")
        
        self.hdd_data = pd.read_csv(hdd_file)
        print(f"Loaded HDD data with {len(self.hdd_data)} rows")
        
        self.cdd_data = pd.read_csv(cdd_file)
        print(f"Loaded CDD data with {len(self.cdd_data)} rows")
        
        self.positioning_data = pd.read_csv(positioning_file)
        print(f"Loaded positioning data with {len(self.positioning_data)} rows")
        
        self._preprocess_data()
    
    def _preprocess_data(self):
        self.price_data['StandardDate'] = self.price_data['Date'].apply(self._standardize_date)
        self.inventory_data['StandardDate'] = self.inventory_data['Date'].apply(self._standardize_date)
        self.hdd_data['StandardDate'] = self.hdd_data['Date'].apply(self._standardize_date)
        self.cdd_data['StandardDate'] = self.cdd_data['Date'].apply(self._standardize_date)
        self.positioning_data['StandardDate'] = self.positioning_data['Date'].apply(self._standardize_date)
        
        self.price_data['StandardDate'] = pd.to_datetime(self.price_data['StandardDate'])
        self.inventory_data['StandardDate'] = pd.to_datetime(self.inventory_data['StandardDate'])
        self.hdd_data['StandardDate'] = pd.to_datetime(self.hdd_data['StandardDate'])
        self.cdd_data['StandardDate'] = pd.to_datetime(self.cdd_data['StandardDate'])
        self.positioning_data['StandardDate'] = pd.to_datetime(self.positioning_data['StandardDate'])
        
        self.price_data.sort_values('StandardDate', inplace=True)
        self.inventory_data.sort_values('StandardDate', inplace=True)
        self.hdd_data.sort_values('StandardDate', inplace=True)
        self.cdd_data.sort_values('StandardDate', inplace=True)
        self.positioning_data.sort_values('StandardDate', inplace=True)
    
    def _standardize_date(self, date_str):
        if not date_str or not isinstance(date_str, str):
            return None
        
        parts = date_str.split('-')
        if len(parts) != 3:
            return date_str
        
        if len(parts[0]) == 2 and len(parts[1]) == 2 and len(parts[2]) == 4:
            return f"{parts[2]}-{parts[1]}-{parts[0]}"
        elif len(parts[0]) == 4 and len(parts[1]) == 2 and len(parts[2]) == 2:
            return date_str
        else:
            return date_str
    
    def merge_datasets(self):
        merged = self.price_data[['StandardDate', 'Date', 'NG1', 'NG2']].copy()
        merged.rename(columns={'Date': 'OriginalDate'}, inplace=True)
        
        inventory_subset = self.inventory_data[['StandardDate', 'Inventory (unit: BCF)']].copy()
        merged = pd.merge(merged, inventory_subset, on='StandardDate', how='left')
        
        hdd_subset = self.hdd_data[['StandardDate', 'HDD']].copy()
        merged = pd.merge(merged, hdd_subset, on='StandardDate', how='left')
        
        cdd_subset = self.cdd_data[['StandardDate', 'CDD']].copy()
        merged = pd.merge(merged, cdd_subset, on='StandardDate', how='left')
        
        positioning_subset = self.positioning_data[['StandardDate', 'Managed Money', 'Producers', 'Swaps']].copy()
        merged = pd.merge(merged, positioning_subset, on='StandardDate', how='left')
        
        merged['Spread'] = merged['NG2'] - merged['NG1']
        
        self.merged_data = merged
        print(f"Created merged dataset with {len(merged)} rows")
        
        return merged
    
    def engineer_features(self):
        if self.merged_data is None:
            print("Error: No merged data available. Run merge_datasets first.")
            return None
        
        data = self.merged_data.copy()
        
        data['PriceChange'] = data['NG1'].diff()
        data['PriceChangePct'] = data['NG1'].pct_change() * 100
        
        data['SpreadChange'] = data['Spread'].diff()
        data['SpreadChangePct'] = data['Spread'].pct_change() * 100
        
        data['MA5'] = data['NG1'].rolling(window=5).mean()
        data['MA20'] = data['NG1'].rolling(window=20).mean()
        
        data['MA5_GT_MA20'] = (data['MA5'] > data['MA20']).astype(int)
        data['MACrossSignal'] = data['MA5_GT_MA20'].diff()
        
        data['InventoryChange'] = data['Inventory (unit: BCF)'].diff()
        data['InventoryChangePct'] = data['Inventory (unit: BCF)'].pct_change() * 100
        
        data['TotalDegreeDays'] = data['HDD'] + data['CDD']
        
        data['WeatherAdjustedInventory'] = data.apply(
            lambda x: x['Inventory (unit: BCF)'] / x['TotalDegreeDays'] if x['TotalDegreeDays'] > 0 else x['Inventory (unit: BCF)'], 
            axis=1
        )
        
        data['ManagedMoneyChange'] = data['Managed Money'].diff()
        data['ManagedMoneyChangePct'] = data['Managed Money'].pct_change() * 100
        
        data['MMPriceRatio'] = data['Managed Money'] / data['NG1']
        
        data['Month'] = data['StandardDate'].dt.month
        data['DayOfWeek'] = data['StandardDate'].dt.dayofweek
        
        data['IsWinter'] = data['Month'].isin([12, 1, 2]).astype(int)
        data['IsSummer'] = data['Month'].isin([6, 7, 8]).astype(int)
        data['IsShoulder'] = (~data['Month'].isin([12, 1, 2, 6, 7, 8])).astype(int)
        
        data['CompositeSignal'] = 0
        
        mask = (data['MA5'].notna() & data['MA20'].notna())
        
        for idx in data[mask].index:
            signal = 0
            
            if data.loc[idx, 'MACrossSignal'] == 1:
                signal += 30
            elif data.loc[idx, 'MACrossSignal'] == -1:
                signal -= 30
            elif data.loc[idx, 'MA5_GT_MA20'] == 1:
                signal += 15
            else:
                signal -= 15
            
            if not np.isnan(data.loc[idx, 'SpreadChange']):
                if data.loc[idx, 'SpreadChange'] < -0.05:
                    signal += 10
                elif data.loc[idx, 'SpreadChange'] > 0.05:
                    signal -= 10
            
            if not np.isnan(data.loc[idx, 'InventoryChange']):
                if data.loc[idx, 'InventoryChange'] < -10:
                    signal += 20
                elif data.loc[idx, 'InventoryChange'] < -5:
                    signal += 10
                elif data.loc[idx, 'InventoryChange'] > 10:
                    signal -= 20
                elif data.loc[idx, 'InventoryChange'] > 5:
                    signal -= 10
            
            if not np.isnan(data.loc[idx, 'HDD']):
                if data.loc[idx, 'HDD'] > 10 and data.loc[idx, 'IsWinter'] == 1:
                    signal += 15
            
            if not np.isnan(data.loc[idx, 'CDD']):
                if data.loc[idx, 'CDD'] > 10 and data.loc[idx, 'IsSummer'] == 1:
                    signal += 10
            
            if not np.isnan(data.loc[idx, 'ManagedMoneyChange']):
                if data.loc[idx, 'ManagedMoneyChange'] > 10000:
                    signal += 15
                elif data.loc[idx, 'ManagedMoneyChange'] > 5000:
                    signal += 7
                elif data.loc[idx, 'ManagedMoneyChange'] < -10000:
                    signal -= 15
                elif data.loc[idx, 'ManagedMoneyChange'] < -5000:
                    signal -= 7
            
            if data.loc[idx, 'Month'] in [1, 12]:
                signal += 10
            elif data.loc[idx, 'Month'] in [7, 8]:
                signal += 5
            elif data.loc[idx, 'Month'] in [4, 5]:
                signal -= 10
            
            data.loc[idx, 'CompositeSignal'] = signal
        
        self.trading_data = data
        print(f"Added trading features to dataset")
        
        return data

class EnhancedNaturalGasStrategy(NaturalGasStrategy):
    def engineer_features(self):
        data = super().engineer_features()
        
        data['Volatility'] = data['NG1'].rolling(window=20).std()
        data['VolatilityRatio'] = data['Volatility'] / data['Volatility'].rolling(window=60).mean()
        
        def calculate_rsi(prices, period=14):
            delta = prices.diff()
            gain = delta.where(delta > 0, 0).rolling(window=period).mean()
            loss = -delta.where(delta < 0, 0).rolling(window=period).mean()
            rs = gain / loss
            return 100 - (100 / (1 + rs))
        
        data['RSI'] = calculate_rsi(data['NG1'])
        
        data['BollingerMiddle'] = data['NG1'].rolling(window=20).mean()
        data['BollingerStd'] = data['NG1'].rolling(window=20).std()
        data['UpperBand'] = data['BollingerMiddle'] + (data['BollingerStd'] * 2)
        data['LowerBand'] = data['BollingerMiddle'] - (data['BollingerStd'] * 2)
        data['BollingerPosition'] = (data['NG1'] - data['LowerBand']) / (data['UpperBand'] - data['LowerBand'])
        
        data['Momentum5d'] = data['NG1'] / data['NG1'].shift(5) - 1
        data['Momentum20d'] = data['NG1'] / data['NG1'].shift(20) - 1
        
        data['SpreadMomentum5d'] = data['Spread'] / data['Spread'].shift(5) - 1
        
        monthly_performance = {}
        for month in range(1, 13):
            monthly_data = data[data['Month'] == month]['PriceChangePct'].dropna()
            monthly_performance[month] = monthly_data.mean() if len(monthly_data) > 0 else 0
        
        data['HistoricalMonthPerformance'] = data['Month'].map(monthly_performance)
        
        mask = (data['MA5'].notna() & data['MA20'].notna() & data['RSI'].notna())
        
        for idx in data[mask].index:
            base_signal = data.loc[idx, 'CompositeSignal']
            
            rsi = data.loc[idx, 'RSI']
            if not np.isnan(rsi):
                if rsi < 30:
                    base_signal += 15
                elif rsi > 70:
                    base_signal -= 15
            
            bb_position = data.loc[idx, 'BollingerPosition']
            if not np.isnan(bb_position):
                if bb_position < 0.2:
                    base_signal += 10
                elif bb_position > 0.8:
                    base_signal -= 10
            
            vol_ratio = data.loc[idx, 'VolatilityRatio']
            if not np.isnan(vol_ratio):
                if vol_ratio > 1.2:
                    base_signal *= 0.8
            
            month_perf = data.loc[idx, 'HistoricalMonthPerformance']
            if not np.isnan(month_perf):
                if month_perf > 0.5:
                    base_signal += 10
                elif month_perf < -0.5:
                    base_signal -= 10
            
            data.loc[idx, 'EnhancedSignal'] = base_signal
        
        self.trading_data = data
        print(f"Added enhanced trading features to dataset")
        
        return data
    
    def backtest_strategy(self, initial_capital=100000, position_size_pct=0.1, use_enhanced_signal=True):
        if self.trading_data is None:
            print("Error: No trading data available. Run engineer_features first.")
            return None
        
        data = self.trading_data.copy()
        
        signal_column = 'EnhancedSignal' if use_enhanced_signal and 'EnhancedSignal' in data.columns else 'CompositeSignal'
        
        capital = initial_capital
        position = 0
        entry_price = 0
        entry_date = None
        trade_history = []
        equity_curve = [initial_capital]
        
        trades = 0
        wins = 0
        losses = 0
        
        for i in range(20, len(data)):
            row = data.iloc[i]
            current_date = row['StandardDate']
            current_price = row['NG1']
            
            if pd.isna(current_price):
                continue
            
            if pd.isna(row[signal_column]):
                continue
            
            signal = row[signal_column]
            
            vol_adjustment = 1.0
            if 'VolatilityRatio' in row and not pd.isna(row['VolatilityRatio']):
                if row['VolatilityRatio'] > 1.5:
                    vol_adjustment = 0.6
                elif row['VolatilityRatio'] < 0.8:
                    vol_adjustment = 1.2
            
            signal_adjustment = 1.0
            signal_strength = abs(signal) / 100
            signal_adjustment = 0.5 + signal_strength
            
            effective_position_size = position_size_pct * vol_adjustment * signal_adjustment
            effective_position_size = min(effective_position_size, 0.25)
            
            if position == 0:
                if signal >= 50:
                    position_value = capital * effective_position_size
                    shares = position_value / current_price
                    
                    position = 1
                    entry_price = current_price
                    entry_date = current_date
                    
                    trade_history.append({
                        'Type': 'Entry',
                        'Direction': 'Long',
                        'Date': current_date,
                        'Price': current_price,
                        'Signal': signal,
                        'PositionSize': effective_position_size,
                        'Shares': shares,
                        'Capital': capital
                    })
                
                elif signal <= -50:
                    position_value = capital * effective_position_size
                    shares = position_value / current_price
                    
                    position = -1
                    entry_price = current_price
                    entry_date = current_date
                    
                    trade_history.append({
                        'Type': 'Entry',
                        'Direction': 'Short',
                        'Date': current_date,
                        'Price': current_price,
                        'Signal': signal,
                        'PositionSize': effective_position_size,
                        'Shares': shares,
                        'Capital': capital
                    })
            
            elif position == 1:
                exit_threshold = -20
                if 'VolatilityRatio' in row and not pd.isna(row['VolatilityRatio']):
                    if row['VolatilityRatio'] > 1.5:
                        exit_threshold = -10
                
                if signal < 0 or signal <= exit_threshold:
                    shares = trade_history[-1]['Shares']
                    pnl = shares * (current_price - entry_price)
                    capital += pnl
                    
                    trade_history.append({
                        'Type': 'Exit',
                        'Direction': 'Long',
                        'Date': current_date,
                        'Price': current_price,
                        'Signal': signal,
                        'Shares': shares,
                        'PnL': pnl,
                        'PercentReturn': (pnl / (shares * entry_price)) * 100,
                        'Capital': capital
                    })
                    
                    trades += 1
                    if pnl > 0:
                        wins += 1
                    else:
                        losses += 1
                    
                    position = 0
                    
                    if signal <= -50:
                        position_value = capital * effective_position_size
                        shares = position_value / current_price
                        
                        position = -1
                        entry_price = current_price
                        entry_date = current_date
                        
                        trade_history.append({
                            'Type': 'Entry',
                            'Direction': 'Short',
                            'Date': current_date,
                            'Price': current_price,
                            'Signal': signal,
                            'PositionSize': effective_position_size,
                            'Shares': shares,
                            'Capital': capital
                        })
            
            elif position == -1:
                exit_threshold = 20
                if 'VolatilityRatio' in row and not pd.isna(row['VolatilityRatio']):
                    if row['VolatilityRatio'] > 1.5:
                        exit_threshold = 10
                
                if signal > 0 or signal >= exit_threshold:
                    shares = trade_history[-1]['Shares']
                    pnl = shares * (entry_price - current_price)
                    capital += pnl
                    
                    trade_history.append({
                        'Type': 'Exit',
                        'Direction': 'Short',
                        'Date': current_date,
                        'Price': current_price,
                        'Signal': signal,
                        'Shares': shares,
                        'PnL': pnl,
                        'PercentReturn': (pnl / (shares * entry_price)) * 100,
                        'Capital': capital
                    })
                    
                    trades += 1
                    if pnl > 0:
                        wins += 1
                    else:
                        losses += 1
                    
                    position = 0
                    
                    if signal >= 50:
                        position_value = capital * effective_position_size
                        shares = position_value / current_price
                        
                        position = 1
                        entry_price = current_price
                        entry_date = current_date
                        
                        trade_history.append({
                            'Type': 'Entry',
                            'Direction': 'Long',
                            'Date': current_date,
                            'Price': current_price,
                            'Signal': signal,
                            'PositionSize': effective_position_size,
                            'Shares': shares,
                            'Capital': capital
                        })
            
            equity_curve.append(capital)
        
        if position != 0:
            last_price = data.iloc[-1]['NG1']
            last_date = data.iloc[-1]['StandardDate']
            
            if not pd.isna(last_price):
                shares = trade_history[-1]['Shares']
                if position == 1:
                    pnl = shares * (last_price - entry_price)
                else:
                    pnl = shares * (entry_price - last_price)
                
                capital += pnl
                
                trade_history.append({
                    'Type': 'Exit',
                    'Direction': 'Long' if position == 1 else 'Short',
                    'Date': last_date,
                    'Price': last_price,
                    'Signal': None,
                    'Shares': shares,
                    'PnL': pnl,
                    'PercentReturn': (pnl / (shares * entry_price)) * 100,
                    'Capital': capital
                })
                
                trades += 1
                if pnl > 0:
                    wins += 1
                else:
                    losses += 1
        
        win_rate = (wins / trades * 100) if trades > 0 else 0
        total_return = ((capital - initial_capital) / initial_capital * 100)
        avg_return_per_trade = (total_return / trades) if trades > 0 else 0
        
        equity_curve = np.array(equity_curve)
        peak = np.maximum.accumulate(equity_curve)
        drawdown = (peak - equity_curve) / peak * 100
        max_drawdown = np.max(drawdown)
        
        if len(trade_history) > 0:
            trade_returns = [trade['PercentReturn'] for trade in trade_history if 'PercentReturn' in trade]
            if trade_returns:
                annual_return = total_return / (len(data) / 252)
                sharpe_ratio = (np.mean(trade_returns) / np.std(trade_returns)) * np.sqrt(252) if np.std(trade_returns) > 0 else 0
            else:
                annual_return = 0
                sharpe_ratio = 0
        else:
            annual_return = 0
            sharpe_ratio = 0
        
        self.trades = trade_history
        self.performance = {
            'initial_capital': initial_capital,
            'final_capital': capital,
            'total_return': total_return,
            'annual_return': annual_return,
            'total_trades': trades,
            'winning_trades': wins,
            'losing_trades': losses,
            'win_rate': win_rate,
            'avg_return_per_trade': avg_return_per_trade,
            'max_drawdown': max_drawdown,
            'sharpe_ratio': sharpe_ratio,
            'equity_curve': equity_curve
        }
        
        print(f"Enhanced Backtest Results:")
        print(f"Total Trades: {trades}")
        print(f"Win Rate: {win_rate:.2f}%")
        print(f"Total Return: {total_return:.2f}%")
        print(f"Annual Return: {annual_return:.2f}%")
        print(f"Avg Return per Trade: {avg_return_per_trade:.2f}%")
        print(f"Max Drawdown: {max_drawdown:.2f}%")
        print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
        
        return self.performance

def run_enhanced_strategy():
    strategy = EnhancedNaturalGasStrategy()
    
    strategy.load_data(
        price_file='Natural_Gas_Data prices.csv',
        inventory_file='inventory.csv',
        hdd_file='HDD.csv',
        cdd_file='CDD.csv',
        positioning_file='Natural_Gas_Data positioning.csv'
    )
    
    strategy.merge_datasets()
    strategy.engineer_features()
    strategy.backtest_strategy(initial_capital=100000, position_size_pct=0.1, use_enhanced_signal=True)
    
    return strategy

if __name__ == "__main__":
    print("Running Enhanced Natural Gas Trading Strategy")
    strategy = run_enhanced_strategy()

Running Enhanced Natural Gas Trading Strategy
Loaded price data with 1621 rows
Loaded inventory data with 6708 rows
Loaded HDD data with 2350 rows
Loaded CDD data with 2350 rows
Loaded positioning data with 334 rows
Created merged dataset with 1621 rows
Added trading features to dataset
Added enhanced trading features to dataset
Enhanced Backtest Results:
Total Trades: 9
Win Rate: 44.44%
Total Return: -1.76%
Annual Return: -0.27%
Avg Return per Trade: -0.20%
Max Drawdown: 2.48%
Sharpe Ratio: -2.64
